In [1]:
import time
import requests
import numpy as np
import pandas as pd
import logging
import datetime
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging

# where API KEY stored.(USE YOUR OWN KEY)
from cert import binanceKey

# my own library
from backTester.BackTester import BackTester
from backTester.GenBackData import BackData
from MyStrategy.MyStrategy import MyStrategy

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# data loading
client = UMFutures()
# backdata 가져오기 (간격 : 5m)
# 2022-08-21 18:30:00 ~ 2022-10-09 09:25:00
first_openTime = 1665156600000 # 2022, 10, 8, 0, 30
df1 = BackData().get_backdata(client, '5m', startTime=first_openTime, limit=1500)
df2 = BackData().get_backdata(client, '5m', startTime=first_openTime-5*1500*60*1000, limit=1500)
df3 = BackData().get_backdata(client, '5m', startTime=first_openTime-2*5*1500*60*1000, limit=1500)
df4 = BackData().get_backdata(client, '5m', startTime=first_openTime-3*5*1500*60*1000, limit=1500)
df5 = BackData().get_backdata(client, '5m', startTime=first_openTime-4*5*1500*60*1000, limit=1500)
df6 = BackData().get_backdata(client, '5m', startTime=first_openTime-5*5*1500*60*1000, limit=1500)
df7 = BackData().get_backdata(client, '5m', startTime=first_openTime-6*5*1500*60*1000, limit=1500)
df8 = BackData().get_backdata(client, '5m', startTime=first_openTime-7*5*1500*60*1000, limit=1500)
df9 = BackData().get_backdata(client, '5m', startTime=first_openTime-8*5*1500*60*1000, limit=1500)
df10 = BackData().get_backdata(client, '5m', startTime=first_openTime-9*5*1500*60*1000, limit=1500)

backdata = pd.concat([df10, df9, df8, df7, df6, df5, df4, df3, df2, df1], axis=0)
backdata.reset_index(inplace=True, drop=True)

In [2]:
backdata = pd.read_pickle('backdata.pkl')

In [3]:
display(backdata.head())
print(backdata.shape)

,OpenTime,Open,High,Low,Close,Volume,CloseTime,QuoteAssetVolume,NumOfTrades,TakerBuyBaseAssetVolume,TakerBuyQuoteAssetVolume
0,2022-08-21 18:30:00,21414.2,21420.9,21400.7,21402.6,489.982,2022-08-21 18:34:59.999,1.049103e+07,5200,222.053,4.754666e+06
1,2022-08-21 18:35:00,21402.6,21450.7,21402.3,21450.7,988.036,2022-08-21 18:39:59.999,2.117409e+07,7545,724.010,1.551653e+07
2,2022-08-21 18:40:00,21450.7,21459.0,21441.0,21457.5,561.106,2022-08-21 18:44:59.999,1.203545e+07,5931,273.934,5.875863e+06
3,2022-08-21 18:45:00,21457.5,21474.9,21442.8,21470.3,708.999,2022-08-21 18:49:59.999,1.521840e+07,6912,368.193,7.903111e+06
4,2022-08-21 18:50:00,21470.3,21486.0,21462.3,21480.5,833.604,2022-08-21 18:54:59.999,1.790326e+07,7757,443.977,9.535305e+06


(14885, 11)


In [4]:
# build Strategy
tester = BackTester()

backdata['MA1'] = tester.get_MA(10, backdata['Close'])
backdata['MA2'] = tester.get_MA(30, backdata['Close'])

myStrg = MyStrategy(3, 0) # i-4, i-3, i-2, i-1, i

myStrg.add_andCondition('MA1', 'MA2', '<', 3)
myStrg.add_andCondition('MA1', 'MA2', '>', 2, func1=lambda x:x*0.9999)
myStrg.add_andCondition('MA1', 'MA2', '>', 1, func1=lambda x:x*0.9998)
myStrg.add_andCondition('MA1', 'MA2', '>', 0, func1=lambda x:x*0.9997)
myStrg.add_Condition('long')

myStrg.add_andCondition('MA1', 'MA2', '>', 3)
myStrg.add_andCondition('MA1', 'MA2', '<', 2, func1=lambda x:x*1.0001)
myStrg.add_andCondition('MA1', 'MA2', '<', 1, func1=lambda x:x*1.0002)
myStrg.add_andCondition('MA1', 'MA2', '<', 0, func1=lambda x:x*1.0003)
myStrg.add_Condition('short')

myStrg.add_andCondition('MA1', 'MA2', '<', 3)
myStrg.add_andCondition('MA1', 'MA2', '>', 2, func1=lambda x:x*0.9999)
myStrg.add_andCondition('MA1', 'MA2', '>', 1, func1=lambda x:x*0.9998)
myStrg.add_andCondition('MA1', 'MA2', '>', 0, func1=lambda x:x*0.9997)
myStrg.add_andCondition('_short_amount', 0, '>', 0)
myStrg.add_Condition('clear')

myStrg.add_andCondition('MA1', 'MA2', '>', 3)
myStrg.add_andCondition('MA1', 'MA2', '<', 2, func1=lambda x:x*1.0001)
myStrg.add_andCondition('MA1', 'MA2', '<', 1, func1=lambda x:x*1.0002)
myStrg.add_andCondition('MA1', 'MA2', '<', 0, func1=lambda x:x*1.0003)
myStrg.add_andCondition('_long_amount', 0, '>', 0)
myStrg.add_Condition('clear')


# myStrg.add_andCondition(lambda x:x, 'MA1', '<', lambda x:x, 'MA2', [3])
# myStrg.add_andCondition(lambda x:x*0.9999, 'MA1', '>', lambda x:x, 'MA2', [2])
# myStrg.add_andCondition(lambda x:x*0.9998, 'MA1', '>', lambda x:x, 'MA2', [1])
# myStrg.add_andCondition(lambda x:x*0.9997, 'MA1', '>', lambda x:x, 'MA2', [0])
# myStrg.add_Condition('long')

# myStrg.add_andCondition(lambda x:x, 'MA1', '>', lambda x:x, 'MA2', [3])
# myStrg.add_andCondition(lambda x:x*1.0001, 'MA1', '<', lambda x:x, 'MA2', [2])
# myStrg.add_andCondition(lambda x:x*1.0002, 'MA1', '<', lambda x:x, 'MA2', [1])
# myStrg.add_andCondition(lambda x:x*1.0003, 'MA1', '<', lambda x:x, 'MA2', [0])
# myStrg.add_Condition('short')

# myStrg.add_andCondition(lambda x:x, 'MA1', '<', lambda x:x, 'MA2', [3])
# myStrg.add_andCondition(lambda x:x*0.9999, 'MA1', '>', lambda x:x, 'MA2', [2])
# myStrg.add_andCondition(lambda x:x*0.9998, 'MA1', '>', lambda x:x, 'MA2', [1])
# myStrg.add_andCondition(lambda x:x*0.9997, 'MA1', '>', lambda x:x, 'MA2', [0])
# myStrg.add_andCondition(lambda x:x, 'short_amount', '>', lambda x:x, 0.0, [0])
# myStrg.add_Condition('clear')

# myStrg.add_andCondition(lambda x:x, 'MA1', '>', lambda x:x, 'MA2', [3])
# myStrg.add_andCondition(lambda x:x*1.0001, 'MA1', '<', lambda x:x, 'MA2', [2])
# myStrg.add_andCondition(lambda x:x*1.0002, 'MA1', '<', lambda x:x, 'MA2', [1])
# myStrg.add_andCondition(lambda x:x*1.0003, 'MA1', '<', lambda x:x, 'MA2', [0])
# myStrg.add_andCondition(lambda x:x, 'long_amount', '>', lambda x:x, 0.0, [0])
# myStrg.add_Condition('clear')

In [5]:
# backtest
tester.backtest_tmp(myStrg, backdata)

#########################
## index 46, short
#########################
long amount:0
long meanPrice:0
short amount:0.0234
short meanPrice:21367.8
asset:1000
maxLimit:499.99348
#########################
## index 72, short
#########################
long amount:0
long meanPrice:0
short amount:0.035
short meanPrice:21398.32457142857
asset:1000
maxLimit:251.05863999999997
#########################
## index 105, clear
#########################
long amount:0
long meanPrice:0
short amount:0
short meanPrice:0
asset:997.2113599999999
maxLimit:997.2113599999999
#########################
## index 105, long
#########################
long amount:0.0232
long meanPrice:21478.0
short amount:0
short meanPrice:0
asset:997.2113599999999
maxLimit:498.92175999999995
#########################
## index 117, clear
#########################
long amount:0
long meanPrice:0
short amount:0
short meanPrice:0
asset:995.26024
maxLimit:995.26024
#########################
## index 117, short
#########################
lo

In [6]:
myStrg.clear_conditions

[[['MA1', 'MA2', '<', None, None, 3],
  ['MA1', 'MA2', '>', <function __main__.<lambda>(x)>, None, 2],
  ['MA1', 'MA2', '>', <function __main__.<lambda>(x)>, None, 1],
  ['MA1', 'MA2', '>', <function __main__.<lambda>(x)>, None, 0],
  ['_short_amount', 0, '>', None, None, 0]],
 [['MA1', 'MA2', '>', None, None, 3],
  ['MA1', 'MA2', '<', <function __main__.<lambda>(x)>, None, 2],
  ['MA1', 'MA2', '<', <function __main__.<lambda>(x)>, None, 1],
  ['MA1', 'MA2', '<', <function __main__.<lambda>(x)>, None, 0],
  ['_long_amount', 0, '>', None, None, 0]]]

In [17]:
tester._make_real_condition(myStrg.clear_conditions[0][4], 105)

False

In [7]:
backdata[50:57][['Close', 'MA1', 'MA2']]

,Close,MA1,MA2
50,21579.9,21443.89,21507.726667
51,21617.5,21452.88,21512.743333
52,21583.8,21472.77,21516.336667
53,21550.2,21496.85,21518.630000
54,21565.3,21522.38,21521.296667
55,21613.2,21548.11,21525.853333
56,21567.6,21568.09,21527.966667


In [ ]:
21496.85*1.001